In [2]:
from graphdriver.load import pancancer, labels
from graphdriver.nets import dataset
from graphdriver.utils import config, constants as cons, ipynb
import matplotlib.pyplot as plt
from tabulate import tabulate
import pandas as pd

In [6]:
ipynb.debug()
cancer = "brca"
conf = config.Conf(cancer)
conf.type = "train"
conf.gcnk = 1
conf.dropouts = 0.3
conf.epochs = 30
conf.weighted = True
conf.use_tumor = True

## Find connection between mutations and driver type


In [4]:
# def avg_connections(data, gcnk: int):
data = dataset.Dataset(conf).data
mut_types = ["silent_fraction","nonsense_fraction", "splice_site_fraction", "missense_fraction", "recurrent_missense_fraction", "frameshift_indel_fraction", "inframe_indel_fraction", "lost_start_stop_fraction", "missense_to_silent", "nonsilent_to_silent", "norm_missense_poisition_entropy", "norm_mutation_entropy"]
df = pd.DataFrame(data.x.numpy(), columns=mut_types)
genes_dict_inv = {v: k for k, v in data.genes_dict.items()}
df.index = df.index.map(genes_dict_inv)
types = [cons.DRIVER_SPEC, cons.DRIVER, cons.CANDIDATE, cons.PASSENGER]
dfs = {}
for t in types:
    # check if t is in data.drivers (passengers as last, only use remaining df)
    if t in data.drivers:
        # only get driver t types
        d = df[df.index.isin(data.drivers[t].index)]
        # remove drivers t from df
        df = df[~df.index.isin(data.drivers[t].index)]
    else:
        d = df
    dfs[t] = d
    
dfs_mean = pd.DataFrame()
for k in dfs:
    if dfs_mean.empty:
        dfs_mean = pd.DataFrame(dfs[k].mean(),columns=[k])
    else:
        dfs_mean[k] = dfs[k].mean()
dfs_mean.plot(figsize=(30,6), use_index=True)
plt.xticks([i for i in range(12)], dfs_mean.index.tolist())
plt.title("Mean mutation feature per gene type")
plt.savefig("brca_stats_mutations.png")
plt.close()

{font_manager.py:1329} DEBUG - findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
{font_manager.py:1341} DEBUG - findfont: score(FontEntry(fname='/home/joe/miniconda3/envs/graph1/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans-Bold.ttf', name='DejaVu Sans', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 0.33499999999999996
{font_manager.py:1341} DEBUG - findfont: score(FontEntry(fname='/home/joe/miniconda3/envs/graph1/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/cmsy10.ttf', name='cmsy10', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
{font_manager.py:1341} DEBUG - findfont: score(FontEntry(fname='/home/joe/miniconda3/envs/graph1/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSerif-Bold.ttf', name='DejaVu Serif', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
{font_

In [8]:
dfs_mean.corr()

,driver_specific,driver,candidate,passenger
driver_specific,1.000000,0.959192,0.944768,0.934794
driver,0.959192,1.000000,0.998633,0.996468
candidate,0.944768,0.998633,1.000000,0.999384
passenger,0.934794,0.996468,0.999384,1.000000
